# Saved States Overview

## View turn pickles

In [2]:
from aquawar import game

# save_dir = "saves/ai_battles/ai_battle_028"
save_dir = "saves/llama3.2_1b_single/llama3.2_1b_single/round_013"

In [23]:
import os
pkl_files = []
for file in os.listdir(save_dir):
    if file.endswith(".pkl"):
        pkl_files.append(file)

print("Found pkl files:", pkl_files)

Found pkl files: ['turn_005.pkl', 'turn_000.pkl', 'latest.pkl', 'turn_006.pkl', 'turn_007.pkl', 'turn_003.pkl']


In [6]:
import pickle
import json
import os
# select_files = pkl_files.copy()
# select_files = ["turn_002.pkl"]
save_dir = ""
select_files = ['good_save/latest.pkl']
select_files = ['demo_tournament_saves/mistral-nemo_12b_S3/gpt-oss_20b_M3/round_002/latest.pkl']

for file in select_files:
    with open(os.path.join(save_dir, file), "rb") as f:
        state = pickle.load(f)
        print(f"=== Loaded state from {file}: ===")
        print(f"Keys:")
        for k, v in state.items():
            if type(v) is dict:
                print(f"  {k}: {[f'{kk}: {type(vv).__name__}' for kk, vv in v.items()]} | {len(v)} items")
            elif type(v) is list and len(v) > 0 and type(v[-1]) is dict:
                print(f"  {k}: list({[f'{kk}: {type(vv).__name__}' for kk, vv in v[-1].items()]}) | {len(v)} items")
        print(json.dumps(state, indent=2))


=== Loaded state from demo_tournament_saves/mistral-nemo_12b_S3/gpt-oss_20b_M3/round_002/latest.pkl: ===
Keys:
  state: ['players: list', 'round_no: int', 'turn_player: int', 'move_history: list', 'game_turn: int', 'player_turn: int', 'phase: str', 'current_player: int'] | 8 items
  history: list(['player: int', 'game_turn: int', 'player_turn: int', 'input_messages: list', 'response: dict', 'valid: bool', 'move: str', 'damage_dealt: int', 'damage_taken: int', 'attempt: int', 'max_attempts: int']) | 13 items
  evaluation: ['players: dict', 'game_status: str'] | 2 items
  players: ['1: list', '2: dict'] | 2 items
{
  "state": {
    "players": [
      {
        "name": "mistral-nemo:12b_single",
        "roster": [
          "Sea Wolf",
          "Manta Ray",
          "Sea Turtle",
          "Octopus",
          "Great White Shark",
          "Hammerhead Shark",
          "Clownfish",
          "Mimic Fish"
        ],
        "team": {
          "fish": [
            {
              "nam

## Voter pickles

In [ ]:
from aquawar.persistent import PersistentGameManager
from aquawar.ai.ollama_majority import OllamaVoter
from aquawar.ai.ollama_player import OllamaPlayer
import pickle

# 1. Load a mid-game turn file
pm = PersistentGameManager('saves')
game = pm.load_game_state('llama3.1_8b_single', 'gpt-oss_20b_single', round_num=6, turn=10)

# 2. Set up a voter
voter = OllamaVoter(voter_index=1, name='Voter1', model='llama3.1:8b')
other_player = OllamaPlayer(name='OtherPlayer', model='gpt-oss:20b')
voter._other_player = other_player
voter.set_game_context(game, player_index=0)

# 3. Run a voter move (simulate 'action' phase)
history_entry, result, context = voter.make_voter_move(
    phase='action',
    save_dir='saves',
    round_num=6
)

# 4. Load and compare files
turn_path = 'saves/llama3.1_8b_single/gpt-oss_20b_single/round_006/turn_010.pkl'
voter_path = 'saves/llama3.1_8b_single/gpt-oss_20b_single/round_006/v1_011.pkl'

with open(turn_path, 'rb') as f:
    turn_data = pickle.load(f)
with open(voter_path, 'rb') as f:
    voter_data = pickle.load(f)

# Compare keys and structure
print('Turn file keys:', turn_data.keys())
print('Voter file keys:', voter_data.keys())
print('Game turn:', turn_data['state']['game_turn'], voter_data['state']['game_turn'])
print('History length:', len(turn_data['history']), len(voter_data['history']))
print('Evaluation:', turn_data['evaluation'], voter_data['evaluation'])

Turn file keys: dict_keys(['state', 'history', 'evaluation', 'players'])
Voter file keys: dict_keys(['state', 'history', 'evaluation', 'players'])
Game turn: 10 10
History length: 10 10
Evaluation: {'players': {'1': {'current_hp': 880, 'damage_dealt': 150, 'damage_taken': 800, 'assertions': {'true': 0, 'false': 2, 'skipped': 0}, 'invalid_moves': {'total': 0, 'by_type': {'invalid_response': 0, 'invalid_parameter': 0, 'invalid_action': 0}}}, '2': {'current_hp': 1250, 'damage_dealt': 320, 'damage_taken': 400, 'assertions': {'true': 1, 'false': 1, 'skipped': 0}, 'invalid_moves': {'total': 0, 'by_type': {'invalid_response': 0, 'invalid_parameter': 0, 'invalid_action': 0}}}}, 'game_status': 'ongoing'} {'players': {'1': {'current_hp': 880, 'damage_dealt': 270, 'damage_taken': 800, 'assertions': {'true': 0, 'false': 2, 'skipped': 0}, 'invalid_moves': {'total': 0, 'by_type': {'invalid_response': 0, 'invalid_parameter': 0, 'invalid_action': 0}}}, '2': {'current_hp': 1130, 'damage_dealt': 320, 'd

# Inference testing

In [29]:
from ollama import chat, ChatResponse
from ollama import Client

client = Client(host="http://localhost:20000")
# response: ChatResponse = chat(model='gpt-oss:20b', messages=[
response: ChatResponse = client.chat(model='gpt-oss:20b', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
], options={
  'temperature': 0.7,
})

In [ ]:
input_tokens = response.model_dump()["prompt_eval_count"]
total_tokens = response.model_dump()["eval_count"]
output_tokens = total_tokens - input_tokens
print("Input tokens:", input_tokens)
print("Output tokens:", output_tokens)
print("Total tokens:", total_tokens)
print(f"=====\n{response.message.content}\n=====")
response.model_dump()

Input tokens: 73
Output tokens: 700
Total tokens: 773
**Short answer:**  
Because the air in the Earth’s atmosphere scatters sunlight, and the scattering is far stronger for short‑wavelength (blue) light than for long‑wavelength (red) light. The net effect is that the sky appears blue most of the time.

---

### What’s actually happening?

1. **Sunlight is white** – it’s a mixture of all visible wavelengths (red, orange, yellow, green, blue, indigo, violet).  
2. **Molecules and very small particles in the air** (mostly nitrogen and oxygen) act like tiny “scattering centers.”  
3. **Rayleigh scattering**: The efficiency with which a particle scatters light scales as \(1/\lambda^4\) (where \(\lambda\) is the wavelength).  
   - Blue light (\(\lambda \approx 450\,\text{nm}\)) is scattered ~10–15 times more strongly than red light (\(\lambda \approx 650\,\text{nm}\)).  
4. **Result**: When you look up, you see more scattered blue photons coming from all directions than any other color. Th

{'model': 'gpt-oss:20b',
 'created_at': '2025-08-25T17:21:35.244164623Z',
 'done': True,
 'done_reason': 'stop',
 'total_duration': 12016743277,
 'load_duration': 146921151,
 'prompt_eval_count': 73,
 'prompt_eval_duration': 45046325,
 'eval_count': 773,
 'eval_duration': 11823796967,
 'message': {'role': 'assistant',
  'content': '**Short answer:**  \nBecause the air in the Earth’s atmosphere scatters sunlight, and the scattering is far stronger for short‑wavelength (blue) light than for long‑wavelength (red) light. The net effect is that the sky appears blue most of the time.\n\n---\n\n### What’s actually happening?\n\n1. **Sunlight is white** – it’s a mixture of all visible wavelengths (red, orange, yellow, green, blue, indigo, violet).  \n2. **Molecules and very small particles in the air** (mostly nitrogen and oxygen) act like tiny “scattering centers.”  \n3. **Rayleigh scattering**: The efficiency with which a particle scatters light scales as \\(1/\\lambda^4\\) (where \\(\\lambd

In [12]:
from langchain_ollama import ChatOllama
model = "gpt-oss:20b"
temperature = 0.1
top_p = 0.9

llm = ChatOllama(
    model=model,
    temperature=temperature,
    top_p=top_p,
    base_url="http://localhost:20000"
)
response = llm.invoke("Tell me something confusing")
print("LLM response:", response)

LLM response: content='I’m a quantum physicist, and my cat, Whisker‑Q, is the most curious particle I’ve ever studied. She loves to chase laser pointers, and each beam is a fleeting photon that collapses her wavefunction into a state of pure kinetic energy. When the red dot flickers, she’s a superposition of “pounce” and “pause,” and the act of observation—my hand with the laser—forces her to choose a trajectory. I’ve measured her acceleration, her reaction time, even her probability distribution across the room. Yet, like any quantum system, she remains unpredictable; the dot’s path is a random walk, and she follows it with a cat’s instinct, not a scientist’s hypothesis. In the end, the lab becomes a playground, and I realize that the most elegant experiment is simply watching Whisker‑Q chase the laser, collapsing uncertainty into a moment of pure, joyous motion.' additional_kwargs={} response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-08-25T17:00:30.475775878Z', 'done': Tr

In [23]:
response.content

AttributeError: 'ChatResponse' object has no attribute 'content'

In [9]:
import json
res_json = json.loads(response.model_dump_json())
print(json.dumps(res_json, indent=2))

{
  "content": "The capital of France is **Paris**.",
  "additional_kwargs": {},
  "response_metadata": {
    "model": "gpt-oss:20b",
    "created_at": "2025-08-25T16:53:32.819204679Z",
    "done": true,
    "done_reason": "stop",
    "total_duration": 121084535413,
    "load_duration": 118970222778,
    "prompt_eval_count": 74,
    "prompt_eval_duration": 1497126924,
    "eval_count": 40,
    "eval_duration": 615635499,
    "model_name": "gpt-oss:20b"
  },
  "type": "ai",
  "name": null,
  "id": "run--b7d3834f-f095-4f48-8239-3af2902ec8f9-0",
  "example": false,
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 74,
    "output_tokens": 40,
    "total_tokens": 114
  }
}
